# Battle of Neighborhoods (Abuja and Lagos)

# Introduction: Business Problem

In this project, we will compare 2 states in Nigeria (Abuja and Lagos) to know where is more suitable for *Business trip and/or Vacation trip*. **Lagos** and **Abuja** are major states in **Nigeria** for business trips or vacations trips.  
We are going to tahe a glimplse on which of the state is more suitable to go on *Business trips or vacation trips*.

## Target Audience
Target audiences for this project does not limit to a person who keeps travelling but everyone. People could simply decide to look for a similar location to do vacation or get the best place in the state to go for vaction or business trip.  
So target for this project is basically everyone who is exploring different places or similar places.

# Data

In order to accomplish this goal, the cities of each state with their latitudes and longitudes are required. The longitude and latitude of each locations in the state were computed manually as well as the cities of each state . From this data, I need to explore the venue category around these cities of Lagos and Abuja by utilizing Foursquare API.

Then I used the acquired location data to explore the venues from the Foursquare API. Using the "explore" option, I look for top 10 venue categories in 10 km radius for each city center and I get "Venue", "Venue Latitude", "Venue Longitude" and "Venue Category" columns.

In [4]:
#!pip install folium

In [11]:
import pandas as pd
import numpy as np

In [12]:
# The code was removed by Watson Studio for sharing.

In [13]:
# The code was removed by Watson Studio for sharing.

In [14]:
# The code was removed by Watson Studio for sharing.

In [15]:
Abuja_Lagos_location_df.head()

,STATE,LGA,District / Area,Latitude,Longitude
0,Abuja,Abaji,Abaji,9.03438,7.47372
1,Abuja,Abaji,Yaba,9.03810,7.48889
2,Abuja,AMAC,Garki,9.03534,7.47703
3,Abuja,AMAC,Gui,9.06146,7.50064
4,Abuja,AMAC,Gwagwa,9.08803,7.31798


In [16]:
Abuja_df.head()

,STATE,LGA,District / Area,Latitude,Longitude
0,Abuja,Abaji,Abaji,9.03438,7.47372
1,Abuja,Abaji,Yaba,9.03810,7.48889
2,Abuja,AMAC,Garki,9.03534,7.47703
3,Abuja,AMAC,Gui,9.06146,7.50064
4,Abuja,AMAC,Gwagwa,9.08803,7.31798


In [17]:
Lagos_df.head()

,STATE,LGA,District / Area,Latitude,Longitude
0,Lagos,Agege,Agege,6.63445,3.30996
1,Lagos,Agege,Orile Agege,6.63098,3.31609
2,Lagos,Ajeromi/Ifelodun,Ajeromi,6.45443,3.32793
3,Lagos,Ajeromi/Ifelodun,Ifelodun,6.45443,3.32793
4,Lagos,Alimosho,Alimosho,6.54599,3.26598


In [18]:
# Import mapping libraries
from geopy.geocoders import Nominatim
import folium

In [19]:
# Importing libraries
#import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [20]:
# The code was removed by Watson Studio for sharing.

In [21]:
address = 'Ekiti'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Nigeria are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Nigeria are 7.736891, 5.2738326.


In [22]:
# create map of Nigeria using latitude and longitude values
map_nigeria = folium.Map(location=[latitude, longitude], zoom_start=8)

# add markers to map
i =0
for lat, lng, borough, neighborhood in zip(Abuja_Lagos_location_df['Latitude'], Abuja_Lagos_location_df['Longitude'], Abuja_Lagos_location_df['LGA'], Abuja_Lagos_location_df['District / Area']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    if Abuja_Lagos_location_df['STATE'][i]=='Abuja':
        color = 'blue'
    else:
        color = 'red'
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=color,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_nigeria)
    i = i+1
    
map_nigeria

# Methodology

In this project we will direct our efforts on detecting the variuos kind of venue category in both states using the **FourSquare API** to get such information

The **Foursquare API** is used to retrieve all venues of each neighborhoods, then group by each neighborhoods and to count how many venues before filter top 10 most common venue types of each neighborhoods and then using the K Means clustering to cluster  each venue Category

In [23]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit=100'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Analysis

Let's perform some basic explanatory data analysis and derive some additional info from our raw data:

In [24]:
Abuja_venues=getNearbyVenues(Abuja_df['District / Area'], Abuja_df['Latitude'], Abuja_df['Longitude'])
Lagos_venues=getNearbyVenues(Lagos_df['District / Area'], Lagos_df['Latitude'], Lagos_df['Longitude'])

In [25]:
Abuja_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abaji,9.03438,7.47372,Area 1 Shopping Complex,9.035810,7.470744,Bookstore
1,Abaji,9.03438,7.47372,Bolton White Hotel,9.034883,7.475637,Hotel Pool
2,Yaba,9.03810,7.48889,Old parade ground,9.042023,7.488829,Soccer Field
3,Yaba,9.03810,7.48889,Agura Hotel,9.039289,7.487925,Resort
4,Yaba,9.03810,7.48889,Blue Light Restaurant and Bar,9.038502,7.491794,Food & Drink Shop


In [26]:
Lagos_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Orile Agege,6.63098,3.31609,Agege Market,6.628480,3.319192,Market
1,Alimosho,6.54599,3.26598,Ikotun market,6.548099,3.266322,Market
2,Alimosho,6.54599,3.26598,Hotel Soluos,6.547196,3.268031,Hotel Pool
3,Alimosho,6.54599,3.26598,Suya spot,6.547192,3.263758,BBQ Joint
4,Alimosho,6.54599,3.26598,Alagomeji,6.543687,3.264760,Bus Station


In [27]:
Abuja_venues.shape

(45, 7)

In [28]:
Lagos_venues.shape

(114, 7)

In [29]:
# Number of venues of each Neighborhood in Abuja
Abuja_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Abaji,2,2,2,2,2,2
Ashara,4,4,4,4,4,4
Bwari,5,5,5,5,5,5
Dafa,1,1,1,1,1,1
Garki,3,3,3,3,3,3
Gumbo,5,5,5,5,5,5
Gwarinpa,3,3,3,3,3,3
Gwaywalada,1,1,1,1,1,1
Karu,3,3,3,3,3,3


In [30]:
# Number of venues of each Neighborhood in lagos
Lagos_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agboyi-Ketu,1,1,1,1,1,1
Alimosho,4,4,4,4,4,4
Amuwo Odofin,4,4,4,4,4,4
Apapa,5,5,5,5,5,5
Badagry,2,2,2,2,2,2
Badagry West,2,2,2,2,2,2
Bariga,2,2,2,2,2,2
Coker Aguda,1,1,1,1,1,1
Egbe-Idimu,4,4,4,4,4,4


In [31]:
# one hot encoding
Abuja_onehot = pd.get_dummies(Abuja_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Abuja_onehot['Neighborhood'] = Abuja_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Abuja_onehot.columns[-1]] + list(Abuja_onehot.columns[:-1])
Abuja_onehot = Abuja_onehot[fixed_columns]

Abuja_onehot.head()

,Neighborhood,Art Gallery,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café,Chinese Restaurant,Construction & Landscaping,Fast Food Restaurant,Food & Drink Shop,Garden Center,Historic Site,Hotel,Hotel Pool,Lounge,Park,Pharmacy,Resort,Shopping Mall,Soccer Field,Turkish Restaurant
0,Abaji,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Abaji,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
2,Yaba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,Yaba,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
4,Yaba,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0


In [32]:
Abuja_grouped = Abuja_onehot.groupby('Neighborhood').mean().reset_index()
Abuja_grouped.head()

,Neighborhood,Art Gallery,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café,Chinese Restaurant,Construction & Landscaping,Fast Food Restaurant,Food & Drink Shop,Garden Center,Historic Site,Hotel,Hotel Pool,Lounge,Park,Pharmacy,Resort,Shopping Mall,Soccer Field,Turkish Restaurant
0,Abaji,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
1,Ashara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.25,0.25,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
2,Bwari,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.00,0.0,0.20,0.2,0.2,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0
3,Dafa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0
4,Garki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.333333,0.333333,0.0,0.0,0.0,0.333333,0.0,0.0


In [33]:
# one hot encoding
Lagos_onehot = pd.get_dummies(Lagos_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Lagos_onehot['Neighborhood'] = Lagos_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Lagos_onehot.columns[-1]] + list(Lagos_onehot.columns[:-1])
Lagos_onehot = Lagos_onehot[fixed_columns]

Lagos_onehot.head()

,Neighborhood,African Restaurant,Airport Terminal,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bistro,Boat or Ferry,Boutique,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,College Cafeteria,Department Store,Electronics Store,Fast Food Restaurant,Film Studio,Fishing Spot,Flea Market,Food Court,Food Truck,Fried Chicken Joint,Heliport,Historic Site,History Museum,Hotel,Hotel Pool,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Lounge,Market,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Nightclub,Optical Shop,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Salon / Barbershop,Shopping Mall,Soccer Field,Soup Place,Supermarket,Theater,Wine Bar,Wine Shop
0,Orile Agege,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Alimosho,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Alimosho,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [34]:
Lagos_grouped = Lagos_onehot.groupby('Neighborhood').mean().reset_index()
Lagos_grouped.head()

,Neighborhood,African Restaurant,Airport Terminal,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bistro,Boat or Ferry,Boutique,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,College Cafeteria,Department Store,Electronics Store,Fast Food Restaurant,Film Studio,Fishing Spot,Flea Market,Food Court,Food Truck,Fried Chicken Joint,Heliport,Historic Site,History Museum,Hotel,Hotel Pool,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Lounge,Market,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Nightclub,Optical Shop,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Salon / Barbershop,Shopping Mall,Soccer Field,Soup Place,Supermarket,Theater,Wine Bar,Wine Shop
0,Agboyi-Ketu,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,Alimosho,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Amuwo Odofin,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Apapa,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0
4,Badagry,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.5,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
# Define a function for top venus for a neighborhood
num_top_venues=10
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Abuja_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Abuja_neighborhoods_venues_sorted['Neighborhood'] = Abuja_grouped['Neighborhood']

for ind in np.arange(Abuja_grouped.shape[0]):
    Abuja_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Abuja_grouped.iloc[ind, :], num_top_venues)

Abuja_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abaji,Bookstore,Hotel Pool,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bus Station,Café
1,Ashara,Hotel,Historic Site,Chinese Restaurant,Fast Food Restaurant,Turkish Restaurant,Construction & Landscaping,BBQ Joint,Bar,Bed & Breakfast,Beer Garden
2,Bwari,Food & Drink Shop,Fast Food Restaurant,Bed & Breakfast,Beer Garden,Garden Center,BBQ Joint,Bar,Bookstore,Bus Station,Café
3,Dafa,Park,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café
4,Garki,Shopping Mall,Lounge,Hotel Pool,Turkish Restaurant,Construction & Landscaping,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore


In [37]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Lagos_neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Lagos_neighborhoods_venues_sorted['Neighborhood'] = Lagos_grouped['Neighborhood']

for ind in np.arange(Lagos_grouped.shape[0]):
    Lagos_neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Lagos_grouped.iloc[ind, :], num_top_venues)

Lagos_neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agboyi-Ketu,Wine Bar,Wine Shop,Breakfast Spot,Food Court,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store
1,Alimosho,Bus Station,BBQ Joint,Hotel Pool,Market,Wine Shop,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store
2,Amuwo Odofin,Fried Chicken Joint,Electronics Store,Beer Garden,Mobile Phone Shop,Breakfast Spot,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Department Store
3,Apapa,Department Store,Pizza Place,Shopping Mall,Hotel,Playground,Bus Station,Business Service,Café,Chinese Restaurant,College Cafeteria
4,Badagry,Museum,History Museum,Food Truck,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store,College Cafeteria


In [38]:
# set number of clusters
kclusters = 5

Abuja_grouped_clustering = Abuja_grouped.drop('Neighborhood', 1)

# run k-means clustering
Abuja_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Abuja_grouped_clustering)

# check cluster labels generated for each row in the dataframe
Abuja_kmeans.labels_[0:10]

array([4, 0, 0, 1, 4, 0, 3, 2, 0, 0], dtype=int32)

In [39]:
Abuja_merged = Abuja_grouped

# add clustering labels
Abuja_merged['Cluster Labels'] = Abuja_kmeans.labels_

# merge Abuja_grouped with Abuja_data to add latitude/longitude for each neighborhood
Abuja_merged = Abuja_merged.join(Abuja_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Abuja_merged.head() # check the last columns!

,Neighborhood,Art Gallery,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café,Chinese Restaurant,Construction & Landscaping,Fast Food Restaurant,Food & Drink Shop,Garden Center,Historic Site,Hotel,Hotel Pool,Lounge,Park,Pharmacy,Resort,Shopping Mall,Soccer Field,Turkish Restaurant,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abaji,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.500000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,4,Bookstore,Hotel Pool,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bus Station,Café
1,Ashara,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.25,0.0,0.0,0.25,0.25,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,Hotel,Historic Site,Chinese Restaurant,Fast Food Restaurant,Turkish Restaurant,Construction & Landscaping,BBQ Joint,Bar,Bed & Breakfast,Beer Garden
2,Bwari,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.0,0.00,0.0,0.20,0.2,0.2,0.00,0.00,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0,Food & Drink Shop,Fast Food Restaurant,Bed & Breakfast,Beer Garden,Garden Center,BBQ Joint,Bar,Bookstore,Bus Station,Café
3,Dafa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.000000,0.000000,1.0,0.0,0.0,0.000000,0.0,0.0,1,Park,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café
4,Garki,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.333333,0.333333,0.0,0.0,0.0,0.333333,0.0,0.0,4,Shopping Mall,Lounge,Hotel Pool,Turkish Restaurant,Construction & Landscaping,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore


In [40]:
# set number of clusters
kclusters = 5

Lagos_grouped_clustering = Lagos_grouped.drop('Neighborhood', 1)

# run k-means clustering
Lagos_kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Lagos_grouped_clustering)

# check cluster labels generated for each row in the dataframe
Lagos_kmeans.labels_[0:10]

array([1, 1, 1, 1, 3, 3, 2, 2, 1, 1], dtype=int32)

In [41]:
Lagos_merged = Lagos_grouped

# add clustering labels
Lagos_merged['Cluster Labels'] = Lagos_kmeans.labels_

# merge Abuja_grouped with Abuja_data to add latitude/longitude for each neighborhood
Lagos_merged = Lagos_merged.join(Lagos_neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Lagos_merged.head() # check the last columns!

,Neighborhood,African Restaurant,Airport Terminal,Art Gallery,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Bed & Breakfast,Beer Garden,Bistro,Boat or Ferry,Boutique,Breakfast Spot,Bus Station,Business Service,Café,Chinese Restaurant,College Cafeteria,Department Store,Electronics Store,Fast Food Restaurant,Film Studio,Fishing Spot,Flea Market,Food Court,Food Truck,Fried Chicken Joint,Heliport,Historic Site,History Museum,Hotel,Hotel Pool,Ice Cream Shop,Indian Restaurant,Italian Restaurant,Lounge,Market,Mobile Phone Shop,Modern European Restaurant,Movie Theater,Museum,Nightclub,Optical Shop,Performing Arts Venue,Pharmacy,Pizza Place,Playground,Plaza,Salon / Barbershop,Shopping Mall,Soccer Field,Soup Place,Supermarket,Theater,Wine Bar,Wine Shop,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agboyi-Ketu,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1,Wine Bar,Wine Shop,Breakfast Spot,Food Court,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store
1,Alimosho,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.25,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Bus Station,BBQ Joint,Hotel Pool,Market,Wine Shop,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store
2,Amuwo Odofin,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,Fried Chicken Joint,Electronics Store,Beer Garden,Mobile Phone Shop,Breakfast Spot,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Department Store
3,Apapa,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.2,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.2,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,1,Department Store,Pizza Place,Shopping Mall,Hotel,Playground,Bus Station,Business Service,Café,Chinese Restaurant,College Cafeteria
4,Badagry,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.5,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,Museum,History Museum,Food Truck,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store,College Cafeteria


In [42]:
Abuja_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abaji,Bookstore,Hotel Pool,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bus Station,Café
1,Ashara,Hotel,Historic Site,Chinese Restaurant,Fast Food Restaurant,Turkish Restaurant,Construction & Landscaping,BBQ Joint,Bar,Bed & Breakfast,Beer Garden
2,Bwari,Food & Drink Shop,Fast Food Restaurant,Bed & Breakfast,Beer Garden,Garden Center,BBQ Joint,Bar,Bookstore,Bus Station,Café
3,Dafa,Park,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café
4,Garki,Shopping Mall,Lounge,Hotel Pool,Turkish Restaurant,Construction & Landscaping,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore
5,Gumbo,Food & Drink Shop,Fast Food Restaurant,Bed & Breakfast,Beer Garden,Garden Center,BBQ Joint,Bar,Bookstore,Bus Station,Café
6,Gwarinpa,Bar,Bed & Breakfast,Café,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Beer Garden,Bookstore,Bus Station,Chinese Restaurant
7,Gwaywalada,Hotel,Turkish Restaurant,Fast Food Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Bus Station,Café
8,Karu,Fast Food Restaurant,Bus Station,Construction & Landscaping,Turkish Restaurant,BBQ Joint,Bar,Bed & Breakfast,Beer Garden,Bookstore,Café
9,Kwali,Food & Drink Shop,Fast Food Restaurant,Bed & Breakfast,Beer Garden,Garden Center,BBQ Joint,Bar,Bookstore,Bus Station,Café


In [43]:
Lagos_neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agboyi-Ketu,Wine Bar,Wine Shop,Breakfast Spot,Food Court,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store
1,Alimosho,Bus Station,BBQ Joint,Hotel Pool,Market,Wine Shop,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store
2,Amuwo Odofin,Fried Chicken Joint,Electronics Store,Beer Garden,Mobile Phone Shop,Breakfast Spot,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Department Store
3,Apapa,Department Store,Pizza Place,Shopping Mall,Hotel,Playground,Bus Station,Business Service,Café,Chinese Restaurant,College Cafeteria
4,Badagry,Museum,History Museum,Food Truck,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store,College Cafeteria
5,Badagry West,Museum,History Museum,Food Truck,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store,College Cafeteria
6,Bariga,Fast Food Restaurant,Boat or Ferry,Wine Shop,Breakfast Spot,Food Court,Flea Market,Fishing Spot,Film Studio,Electronics Store,Department Store
7,Coker Aguda,Fast Food Restaurant,Wine Shop,Breakfast Spot,Food Court,Flea Market,Fishing Spot,Film Studio,Electronics Store,Department Store,College Cafeteria
8,Egbe-Idimu,Fried Chicken Joint,Movie Theater,Shopping Mall,Bus Station,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant,Electronics Store,Department Store
9,Eredo,Market,Electronics Store,Beach,Pharmacy,Wine Shop,Breakfast Spot,Flea Market,Fishing Spot,Film Studio,Fast Food Restaurant


# Results and Discussion

* Lagos has more venues than Abuja ( 114 vs 45)  
* Victoria Island has the most venues in Lagos  
* Bwari and Nyanya has the most venues in Abuja  
* Most of the venues in Victoria Island are Pizza Place and Restaurant  
* Venues in Nyanya and Bwari are more diverse includes: Pharmacy, Building and Construction, restaurants et.c  
* Bars, Restaurants, Bakeries, and Diners are very common in Abuja  
* Lagos is more about commercial and also include Bars , Restaurants et.c  


# Conclusion

* For travelers:  
 Both states, Lagos and Abuja is more suitable to spend a good amount time of enjoy the local food

* For business owners:  
Lagos is good place to open small business related to food and travel industry  
Lagos is better place to involve in industry  
Abuja is better place to open a firm
